In [4]:
from Bio import SeqIO
from Bio.Seq import Seq

def pipeline_adn_to_protein(adn_5a3):
    # Función para mostrar mensajes paso a paso
    def titulo(txt):
        print("\n" + "="*70)
        print(txt)
        print("="*70)

    titulo("Codificante")
    print("Hebra coificante:", adn_5a3[:60], ("..." if len(adn_5a3) > 60 else ""))

    
    # 1. Replicación: Complementaria y Reversa complementaria
    titulo("Replicación")
    complementaria_5a3 = adn_5a3.complement()  # Hebra complementaria (5'->3')
    rev_comp_5a3 = adn_5a3.reverse_complement()  # Hebra reversa complementaria (5'->3')

    # Mostrar resultados
    print("Complementaria (5'->3'):", complementaria_5a3[:60], ("..." if len(complementaria_5a3) > 60 else ""))
    print("Reversa complementaria (5'->3'):", rev_comp_5a3[:60], ("..." if len(rev_comp_5a3) > 60 else ""))

    # Hebra molde 3'->5' para la transcripción
    hebra_molde_3a5 = rev_comp_5a3[::-1]
    print("Hebra molde (3'->5'):", hebra_molde_3a5[:60], ("..." if len(hebra_molde_3a5) > 60 else ""))

    # 2. Transcripción: Generamos el ARNm a partir de la hebra molde
    titulo("Transcripción")
    mrna_5a3 = hebra_molde_3a5.complement().transcribe()  # Convertir 3'->5' a ARNm 5'->3'

    # Mostrar resultados
    print("ARNm (5'->3'):", mrna_5a3[:90], ("..." if len(mrna_5a3) > 90 else ""))

    # 3. Traducción: Desde el primer AUG hasta el primer Stop
    titulo("Traducción")
    
    m = str(mrna_5a3)
    i = m.find("AUG")
    if i == -1:
        print("No hay AUG en el ARNm. No se puede iniciar traducción canónica.")
        prot_desde_AUG = None
    else:
        prot_desde_AUG = Seq(m[i:]).translate(to_stop=True)
        print("Posición del primer AUG:", i)
        # print("Proteína (desde primer AUG hasta Stop):", prot_desde_AUG)

    # Traducción naïve (marco 0)
    prot_naive = mrna_5a3.translate()
    # print("Proteína (naïve, marco 0; puede incluir '*'):", prot_naive)

    # Diagnóstico básico
    if prot_desde_AUG:
        if "*" in str(prot_naive):
            print("Aviso: se observan codones Stop en el recorrido naïve (esperable fuera de la ORF).")
        # print("Longitud proteína canónica:", len(prot_desde_AUG))
    else:
        print("Sugerencia: revisa la orientación de la hebra de entrada o usa otra región con AUG.")

    return prot_desde_AUG, prot_naive

In [5]:
# Ejemplo de ADN 5'->3'
# adn_5a3 = Seq("ATGCCTGAATGC")
adn_5a3 = Seq("ATGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG")

proteina_canónica, proteina_naive = pipeline_adn_to_protein(adn_5a3)

# Proteínas resultantes
print("Proteína canónica (desde primer AUG):", proteina_canónica)
print("Proteína naïve (marco 0):", proteina_naive)



Codificante
Hebra coificante: ATGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG 

Replicación
Complementaria (5'->3'): TACCGGTAACATTACCCGGCGACTTTCCCACGGGCTATC 
Reversa complementaria (5'->3'): CTATCGGGCACCCTTTCAGCGGCCCATTACAATGGCCAT 
Hebra molde (3'->5'): TACCGGTAACATTACCCGGCGACTTTCCCACGGGCTATC 

Transcripción
ARNm (5'->3'): AUGGCCAUUGUAAUGGGCCGCUGAAAGGGUGCCCGAUAG 

Traducción
Posición del primer AUG: 0
Aviso: se observan codones Stop en el recorrido naïve (esperable fuera de la ORF).
Proteína canónica (desde primer AUG): MAIVMGR
Proteína naïve (marco 0): MAIVMGR*KGAR*


**Proteína canónica vs. Proteína naïve**

La **proteína canónica** se obtiene mediante la traducción del **ARNm** desde el primer codón **AUG** (inicio de la traducción) hasta el primer codón **Stop** (final de la traducción). Este proceso sigue las reglas naturales de la **traducción** en la célula, donde solo se traduce la secuencia dentro del **marco de lectura abierto (ORF)**, es decir, la región que contiene el codón **AUG** y finaliza en el primer codón **Stop**.

Por otro lado, la **proteína naïve** se genera traduciendo todo el **ARNm** sin tener en cuenta los codones de inicio y paro. En este caso, el proceso no busca el primer **AUG**, sino que traduce la secuencia completa como si fuera un único bloque continuo, lo que puede resultar en una proteína truncada o con **codones de parada** internos, dependiendo de la secuencia.

* La **proteína canónica** es útil para obtener una versión biológicamente relevante de la proteína, ya que sigue las reglas naturales de la célula y respeta el **marco de lectura**.
* La **proteína naïve**, por su parte, se utiliza principalmente en contextos experimentales o de análisis, ya que permite obtener una traducción rápida y completa del **ARNm**. También se usa cuando se quiere estudiar el **marco completo** de lectura sin importar si hay codones **Stop** en el camino.

Ambas versiones son útiles dependiendo del objetivo: mientras que la **canónica** es la forma funcional de la proteína, la **naïve** ayuda a analizar el ARNm sin restricciones y puede ser útil para obtener una **visión general** del proceso de traducción.
